In [1]:
%matplotlib tk
import numpy as np
import hyperspy.api as hs
import pycrystem as pc
import pymatgen as pmg
from matplotlib import pyplot as plt
from pymatgen.transformations.standard_transformations import RotationTransformation
from pycrystem.indexation_generator import IndexationGenerator
from pycrystem.utils.expt_utils import radial_average
from scipy.constants import pi

In [2]:
si = pmg.Element("Si")
lattice = pmg.Lattice.cubic(5.431)
silicon = pmg.Structure.from_spacegroup("Fd-3m",lattice, [si], [[0, 0, 0]])

In [3]:
size = 256
radius=1.5
ediff = pc.ElectronDiffractionCalculator(300., 0.025)

rotaxis = [0, 0, 1]
thetas = np.arange(0, 45, 1)

data_silicon = []
for theta in thetas:
    rot = RotationTransformation(rotaxis, theta)
    sieg = rot.apply_transformation(silicon)
    diff_dat = ediff.calculate_ed_data(sieg, radius)
    dpi = diff_dat.as_signal(256, 0.03, 1.2)
    data_silicon.append(dpi.data)
    
data = [data_silicon]
test_data = pc.ElectronDiffraction(data)
test_data.set_calibration(1.2/128)

In [77]:
test_data.plot()

In [4]:
rot_list = []
nstep=119
for i in np.arange(nstep):
    theta = (i*59.5/(nstep-1))/180*pi
    rot_list.append((theta, 0., 0.))

In [11]:
rot_list[50]

(0.44003004446467203, 0.0, 0.0)

In [5]:
edc = pc.ElectronDiffractionCalculator(300, 0.025)
diff_gen = pc.DiffractionLibraryGenerator(edc)
struc_lib = dict()
struc_lib['si'] = (silicon, rot_list)
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=1.2/128,
                                            reciprocal_radius=1.5,
                                            representation='euler')

In [6]:
### Ground Truth
indexer = IndexationGenerator(test_data, library)
match_results = indexer.correlate()

A Jupyter Widget

In [16]:
radial_sims = {}
for XZX in library['si']:
    peaks_2D_data = library['si'][XZX].as_signal(256,0,1.2).data #basically a local
    radial_sims[XZX] = radial_average(peaks_2D_data,center=[256/2,256/2],cython=False)

In [14]:
radial_averaged_data = test_data.map(radial_average,center=np.asarray([256/2,256/2]),cython=False,inplace=False, show_progressbar=False)

In [15]:
for XZX in radial_sims:
    #plt.plot(radial_sims[XZX])
    break
plt.plot(radial_averaged_data.inav[0,0],color='k')
plt.plot(radial_sims[(0.0,0.00,0.00)],color='r')
plt.plot(radial_sims[rot_list[50]],color='b')

In [17]:
def mapable_library_matcher(dp_radial,dict_of_radial_patterns,n_scorer=5):
    best_corr = [0]*n_scorer 
    best_corr_key = [None]*n_scorer
    from pycrystem.utils import _correlate
    for key in dict_of_radial_patterns:
        radial_pattern_rotate = dict_of_radial_patterns[key]
        correlation = _correlate(dp_radial,radial_pattern_rotate)
        if correlation > np.min(best_corr):
            best_corr[np.argmin(best_corr)] = correlation
            best_corr_key[np.argmin(best_corr)] = key
            
    return best_corr,best_corr_key

In [18]:
alpha = radial_averaged_data.map(mapable_library_matcher,dict_of_radial_patterns=radial_sims,n_scorer=2,inplace=False)

A Jupyter Widget

In [20]:
alpha.inav[0]

([0.76828670512207153, 0.76824999083863821],
 [(0.18481261867516227, 0.0, 0.0), (0.299220430235977, 0.0, 0.0)])